# Index Comparison Guide

This notebook compares different indexing algorithms available in VectorDB and helps you choose the right one for your use case.

## Index Types

| Index | Best For | Speed | Recall | Memory |
|-------|----------|-------|--------|--------|
| **Flat** | Small datasets (<10k) | ⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| **IVF** | Medium datasets | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **HNSW** | General purpose | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐ |
| **PQ** | Large datasets, memory constrained | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ |

In [ ]:
import numpy as np
import time
from typing import List, Dict, Any
import matplotlib.pyplot as plt

from vectordb import VectorDatabase
from vectordb.index import FlatIndex, IVFIndex, HNSWIndex, ProductQuantizationIndex

# For computing ground truth
from scipy.spatial.distance import cdist

np.random.seed(42)
print("Setup complete!")

## Generate Test Data

In [ ]:
# Dataset configuration
N_VECTORS = 10000  # Number of vectors in the database
N_QUERIES = 100    # Number of query vectors
DIMENSION = 128    # Vector dimension
K = 10             # Number of neighbors to retrieve

# Generate random vectors
print(f"Generating {N_VECTORS} database vectors...")
data = np.random.randn(N_VECTORS, DIMENSION).astype(np.float32)

print(f"Generating {N_QUERIES} query vectors...")
queries = np.random.randn(N_QUERIES, DIMENSION).astype(np.float32)

# Generate IDs
ids = [f"vec_{i}" for i in range(N_VECTORS)]

print(f"Data shape: {data.shape}")
print(f"Queries shape: {queries.shape}")

## Compute Ground Truth

We'll use brute-force search to compute the true nearest neighbors for evaluation.

In [ ]:
def compute_ground_truth(data: np.ndarray, queries: np.ndarray, k: int) -> np.ndarray:
    """Compute exact k-nearest neighbors using brute force."""
    distances = cdist(queries, data, metric='euclidean')
    indices = np.argsort(distances, axis=1)[:, :k]
    return indices

def compute_recall(predicted: List[List[int]], ground_truth: np.ndarray, k: int) -> float:
    """Compute recall@k."""
    recalls = []
    for pred, gt in zip(predicted, ground_truth):
        pred_set = set(pred[:k])
        gt_set = set(gt[:k].tolist())
        recall = len(pred_set & gt_set) / k
        recalls.append(recall)
    return np.mean(recalls)

print("Computing ground truth...")
ground_truth = compute_ground_truth(data, queries, K)
print(f"Ground truth shape: {ground_truth.shape}")
print(f"Sample ground truth for query 0: {ground_truth[0]}")

## 1. Flat Index (Brute Force)

The Flat index performs exact nearest neighbor search by comparing the query to every vector in the database.

**Pros:**
- 100% recall (exact results)
- No training required
- Simple and reliable

**Cons:**
- O(n) search time
- Too slow for large datasets

In [ ]:
# Create and build Flat index
print("Building Flat index...")
flat_index = FlatIndex(dimension=DIMENSION, metric='euclidean')

build_start = time.time()
flat_index.add(data, ids=ids)
flat_build_time = time.time() - build_start

print(f"Build time: {flat_build_time:.3f}s")
print(f"Vectors indexed: {flat_index.count()}")

In [ ]:
# Benchmark Flat index search
def benchmark_index(index, queries, k, n_runs=3):
    """Benchmark search performance."""
    times = []
    all_results = None
    
    for _ in range(n_runs):
        start = time.time()
        results = []
        for query in queries:
            result = index.search(query, k=k)
            # Extract indices from results
            indices = [int(r['id'].split('_')[1]) for r in result]
            results.append(indices)
        times.append(time.time() - start)
        all_results = results
    
    return {
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'qps': len(queries) / np.mean(times),
        'results': all_results
    }

flat_benchmark = benchmark_index(flat_index, queries, K)
flat_recall = compute_recall(flat_benchmark['results'], ground_truth, K)

print(f"Flat Index Results:")
print(f"  Search time: {flat_benchmark['mean_time']:.3f}s (±{flat_benchmark['std_time']:.3f})")
print(f"  QPS: {flat_benchmark['qps']:.1f}")
print(f"  Recall@{K}: {flat_recall:.4f}")

## 2. IVF Index (Inverted File)

IVF partitions the vector space into clusters. During search, only a subset of clusters (nprobe) are searched.

**Pros:**
- Much faster than flat for large datasets
- Tunable speed/recall tradeoff (nprobe)

**Cons:**
- Requires training on data
- Lower recall than flat (approximate)

In [ ]:
# Create and build IVF index
N_CLUSTERS = 100  # Number of clusters

print(f"Building IVF index with {N_CLUSTERS} clusters...")
ivf_index = IVFIndex(dimension=DIMENSION, n_clusters=N_CLUSTERS, metric='euclidean')

build_start = time.time()
ivf_index.train(data)  # IVF requires training
ivf_index.add(data, ids=ids)
ivf_build_time = time.time() - build_start

print(f"Build time: {ivf_build_time:.3f}s")
print(f"Vectors indexed: {ivf_index.count()}")

In [ ]:
# Benchmark IVF with different nprobe values
nprobe_values = [1, 4, 8, 16, 32]
ivf_results = []

print("IVF Index Results:")
print(f"{'nprobe':>8} {'Time (s)':>10} {'QPS':>10} {'Recall':>10}")
print("-" * 42)

for nprobe in nprobe_values:
    ivf_index.nprobe = nprobe
    benchmark = benchmark_index(ivf_index, queries, K)
    recall = compute_recall(benchmark['results'], ground_truth, K)
    
    ivf_results.append({
        'nprobe': nprobe,
        'time': benchmark['mean_time'],
        'qps': benchmark['qps'],
        'recall': recall
    })
    
    print(f"{nprobe:>8} {benchmark['mean_time']:>10.3f} {benchmark['qps']:>10.1f} {recall:>10.4f}")

## 3. HNSW Index (Hierarchical Navigable Small World)

HNSW builds a hierarchical graph structure for efficient approximate nearest neighbor search.

**Pros:**
- Excellent speed/recall tradeoff
- No training required
- Works well for most use cases

**Cons:**
- Higher memory usage (stores graph structure)
- Slower build time

In [ ]:
# Create and build HNSW index
M = 16  # Number of connections per node
EF_CONSTRUCTION = 100  # Size of dynamic candidate list during construction

print(f"Building HNSW index (M={M}, ef_construction={EF_CONSTRUCTION})...")
hnsw_index = HNSWIndex(
    dimension=DIMENSION,
    M=M,
    ef_construction=EF_CONSTRUCTION,
    metric='euclidean'
)

build_start = time.time()
hnsw_index.add(data, ids=ids)
hnsw_build_time = time.time() - build_start

print(f"Build time: {hnsw_build_time:.3f}s")
print(f"Vectors indexed: {hnsw_index.count()}")

In [ ]:
# Benchmark HNSW with different ef_search values
ef_search_values = [16, 32, 64, 128, 256]
hnsw_results = []

print("HNSW Index Results:")
print(f"{'ef_search':>10} {'Time (s)':>10} {'QPS':>10} {'Recall':>10}")
print("-" * 44)

for ef_search in ef_search_values:
    hnsw_index.ef_search = ef_search
    benchmark = benchmark_index(hnsw_index, queries, K)
    recall = compute_recall(benchmark['results'], ground_truth, K)
    
    hnsw_results.append({
        'ef_search': ef_search,
        'time': benchmark['mean_time'],
        'qps': benchmark['qps'],
        'recall': recall
    })
    
    print(f"{ef_search:>10} {benchmark['mean_time']:>10.3f} {benchmark['qps']:>10.1f} {recall:>10.4f}")

## 4. Product Quantization (PQ)

PQ compresses vectors by dividing them into subvectors and quantizing each subvector separately.

**Pros:**
- Significant memory reduction
- Fast search on compressed data

**Cons:**
- Lower recall due to compression
- Requires training

In [ ]:
# Create and build PQ index
N_SUBVECTORS = 16  # Number of subvectors (must divide dimension)
N_BITS = 8  # Bits per subvector (256 centroids)

print(f"Building PQ index (n_subvectors={N_SUBVECTORS}, n_bits={N_BITS})...")
pq_index = ProductQuantizationIndex(
    dimension=DIMENSION,
    n_subvectors=N_SUBVECTORS,
    n_bits=N_BITS,
    metric='euclidean'
)

build_start = time.time()
pq_index.train(data)
pq_index.add(data, ids=ids)
pq_build_time = time.time() - build_start

print(f"Build time: {pq_build_time:.3f}s")
print(f"Vectors indexed: {pq_index.count()}")

In [ ]:
# Benchmark PQ index
pq_benchmark = benchmark_index(pq_index, queries, K)
pq_recall = compute_recall(pq_benchmark['results'], ground_truth, K)

print(f"PQ Index Results:")
print(f"  Search time: {pq_benchmark['mean_time']:.3f}s")
print(f"  QPS: {pq_benchmark['qps']:.1f}")
print(f"  Recall@{K}: {pq_recall:.4f}")

## Comparison Summary

In [ ]:
# Collect all results
summary = [
    {'Index': 'Flat', 'Build Time': flat_build_time, 'QPS': flat_benchmark['qps'], 'Recall': flat_recall},
    {'Index': 'IVF (nprobe=8)', 'Build Time': ivf_build_time, 'QPS': ivf_results[2]['qps'], 'Recall': ivf_results[2]['recall']},
    {'Index': 'HNSW (ef=64)', 'Build Time': hnsw_build_time, 'QPS': hnsw_results[2]['qps'], 'Recall': hnsw_results[2]['recall']},
    {'Index': 'PQ', 'Build Time': pq_build_time, 'QPS': pq_benchmark['qps'], 'Recall': pq_recall},
]

print("\n" + "=" * 70)
print("INDEX COMPARISON SUMMARY")
print("=" * 70)
print(f"{'Index':<20} {'Build Time':>12} {'QPS':>12} {'Recall':>12}")
print("-" * 70)
for row in summary:
    print(f"{row['Index']:<20} {row['Build Time']:>12.3f} {row['QPS']:>12.1f} {row['Recall']:>12.4f}")

In [ ]:
# Visualize recall vs QPS tradeoff
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# IVF tradeoff
ivf_qps = [r['qps'] for r in ivf_results]
ivf_recall = [r['recall'] for r in ivf_results]
ax1.plot(ivf_qps, ivf_recall, 'o-', label='IVF', markersize=8)
for r in ivf_results:
    ax1.annotate(f"np={r['nprobe']}", (r['qps'], r['recall']), textcoords="offset points", xytext=(5,5))

# HNSW tradeoff
hnsw_qps = [r['qps'] for r in hnsw_results]
hnsw_recall = [r['recall'] for r in hnsw_results]
ax1.plot(hnsw_qps, hnsw_recall, 's-', label='HNSW', markersize=8)
for r in hnsw_results:
    ax1.annotate(f"ef={r['ef_search']}", (r['qps'], r['recall']), textcoords="offset points", xytext=(5,5))

# Flat baseline
ax1.axhline(y=flat_recall, color='g', linestyle='--', label='Flat (baseline)')

ax1.set_xlabel('Queries per Second (QPS)')
ax1.set_ylabel(f'Recall@{K}')
ax1.set_title('Recall vs Speed Tradeoff')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Build time comparison
indices = ['Flat', 'IVF', 'HNSW', 'PQ']
build_times = [flat_build_time, ivf_build_time, hnsw_build_time, pq_build_time]
ax2.bar(indices, build_times, color=['blue', 'orange', 'green', 'red'])
ax2.set_ylabel('Build Time (seconds)')
ax2.set_title('Index Build Time Comparison')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Choosing the Right Index

### Decision Guide

```
Dataset Size?
├── < 10,000 vectors → Use FLAT (exact results, fast enough)
├── 10k - 1M vectors
│   ├── Memory constrained? → Use IVF-PQ
│   └── Otherwise → Use HNSW
└── > 1M vectors
    ├── Need highest recall? → Use IVF-HNSW (hybrid)
    └── Memory constrained? → Use IVF-PQ
```

### Key Takeaways

1. **HNSW** is the best general-purpose index for most applications
2. **IVF** is good when you need to tune the speed/recall tradeoff dynamically
3. **PQ** is essential when memory is a constraint
4. **Flat** is perfect for small datasets or when you need 100% recall

## Next Steps

- **03_performance_tuning.ipynb**: Learn how to tune index parameters for optimal performance